We want to start a cluster with MPI capability.  
Bottom line is, you need to have a MPI capable cluster first.  
Unfortunatly, my local machines do NOT support this.  
MS provides a way to bulid cluster machine through MS HPC pack.  
However, the head node needs to run Windows Server OS, which I don't have.  
So, it can be done. 

* 1 Set up the MPI cluster in ipcluster_config.py  
You specify 'cluster_id' and 'engine_launcher_class'

c.BaseParallelApplication.cluster_id = 'mpi'  
c.IPClusterEngines.engine_launcher_class = 'MPIEngineSetLauncher'  

* 2 Launch the MPI cluster  

ipcluster start -n 3 --profile=remote_mpi --ip=140.117.35.xxx

In [13]:
import ipyparallel as ipp
import numpy as np
c = ipp.Client(profile='remote_mpi', cluster_id='mpi')
print(c.ids)

[0, 1, 2, 5, 6]


* 3 Start more engines  
The following will start two engines which belong to their own   
MPI_COMM_WORLD.

mpiexec -n 2 ipengine --profile=remote_mpi   
  --file=C:\Users\adm\.ipython\profile_remote_mpi\security\ipcontroller-mpi-engine.json

If you use ipengine --profile=remote_mpi ..., then you will have one  
engine and an independent MPI_COMM_WORLD, no matter the new eigine(s)  
is in the same machine which started the cluster or not.  
  
Document says that you could also specify in ipengine_config.py, e.g.

c.IPEngineApp.url_file = 'C:\Users\adm\.ipython\profile_remote_mpi\security\ipcontroller-mpi-engine.json'  

Note that ipcontroller-mpi-engine.json is from the local machine when mpi cluster is launched.  
Pay attention to the name, it has "mpi" in it.  

BUT IT DOES NOT WORK FOR ME!

If you don't wish to specify the location od the xxx.json file in the command line when use mpiexec,  
you will have to modify (at least in my case) the xxx.json file name to ipengine_config.py, no "mpi".




In [7]:
c = ipp.Client(profile='remote_mpi', cluster_id='mpi')
print(c.ids)

[0, 1, 2, 3, 4]


We have now 5 engines!

In [8]:
dview = c[:]
dview.scatter('a',np.arange(32,dtype='float64'))

<AsyncResult: scatter>

In [9]:
dview['a']

[array([0., 1., 2., 3., 4., 5., 6.]),
 array([ 7.,  8.,  9., 10., 11., 12., 13.]),
 array([14., 15., 16., 17., 18., 19.]),
 array([20., 21., 22., 23., 24., 25.]),
 array([26., 27., 28., 29., 30., 31.])]

In [10]:
dview.run('psum.py')

<AsyncResult: execute>

In [11]:
%px totalsum = psum(a)

In [12]:
dview['totalsum']

[array(190.), array(190.), array(190.), array(306.), array(306.)]